<a href="https://colab.research.google.com/github/KeaganChasenski/PDF_EXCEL_reading_Daysheet/blob/main/Skukuzas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pdfplumber
!pip install pandas
!pip install matplotlib
!pip install openpyxl
!pip install xlrd

  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (13.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
seaborn 0.12.2 requires matplotlib!=3.6.1,>=3.1, which is not installed.
datascience 0.17.6 requires matplotlib>=3.0.0, which is not installed.
mlxtend 0.23.4 requires matplotlib>=3.0.0, which is not installed.
plotnine 0.14.5 requires matplotlib>=3.8.0, which is not installed.
arviz 0.21.0 requires matplotlib>=3.5, which is not installed.
bigframes 2.3.0 requires matplotlib>=3.7.1, which is not installed.
missingno 0.5.2 requires matplotlib, which is not installed.
geemap 0.35.3 requires matplotlib, which is not installed.
fastai 2.7.19 requires matplotlib, which is not installed.
prophet 1.1.6 requires matplotlib>=2.

In [ ]:
import logging
import pdfplumber
import os
import shutil
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import calendar
import re

from IPython import get_ipython
from IPython.display import display


Code for reading in PDFs

In [ ]:
pdf_folder_path = 'unread_PDFs'
pdf_read_folder_path = 'read_PDFs'

In [ ]:
logging.getLogger("pdfminer").setLevel(logging.ERROR)
all_data = {}

for filename in os.listdir(pdf_folder_path):
  if filename.endswith(".pdf"):
    filepath = os.path.join(pdf_folder_path, filename)
    try:
      with pdfplumber.open(filepath) as pdf:
        # Example: Extracting tables from the first page
        first_page = pdf.pages[0]
        date = pdf.pages[0].extract_text().split('\n')[1]
        #print(date)
        tables = first_page.extract_tables()

        #print(tables)
        all_data[filename] = {"date": date, "tables": tables}
        print(f"Processed {filename}")

        #move the PDF to read folder
        read_folder_path = 'read_PDFs'

        new_filepath = os.path.join(read_folder_path, filename)
        shutil.move(filepath, new_filepath)
        print(f"Moved {filename} to {read_folder_path}")

    except Exception as e:
      print(f"Error processing {filename}: {e}")

In [ ]:

slt_counts = {}

for filename, data in all_data.items():
    file_date = data.get("date")

    file_tables = data.get("tables")
    print(f"Processing tables from {file_date}:")

    slt_counts[filename] = {}

    for i, table in enumerate(file_tables):

      df = pd.DataFrame(table)

      #display(df)

      # Count 'SLT' in column 9 (index 8) and column 18 (index 17)
      arrival_count = 0
      depart_count = 0

      arrival_count = df[df.columns[8]].astype(str).str.lower().str.count('slt').sum()

      depart_count = df[df.columns[17]].astype(str).str.lower().str.count('slt').sum()

      # Store the counts for this specific table
      slt_counts[filename][f"table_{i+1}"] = {
                      "arrival_count": int(arrival_count),
                      "depart_count": int(depart_count)
                    }

      print(f"Number of rooms on and SLT for {file_date} - Arrivals: {arrival_count}, Departures: {depart_count}")


In [ ]:
print("\n--- Final SLT Counts ---")
for filename, tables_data in slt_counts.items():
  print(f"File: {filename}")
  for table_name, counts in tables_data.items():
    print(f"   Arrivals = {counts['arrival_count']}, Departures= {counts['depart_count']}")


--- Final SLT Counts ---


In [ ]:

# --- Calculate Total Daily Counts ---
daily_total_counts = {}

for filename, tables_data in slt_counts.items():
    total_arrivals_for_day = 0
    total_departures_for_day = 0

    for table_name, counts in tables_data.items():
        total_arrivals_for_day += counts.get("arrival_count", 0)
        total_departures_for_day += counts.get("depart_count", 0)

    date_str = all_data.get(filename, {}).get("date")

    if date_str:
        #print(f"Original date string for {filename}: '{date_str}'") # Optional: Keep for debugging

        try:

            match = re.match(r'(\w+)\s+(\d+)(?:th|rd|nd|st)?\s+(\w+)\s+(\d{4})', date_str)

            if match:
                weekday, day, month, year = match.groups()

                date_str_standard = f"{weekday} {day} {month} {year}"

                file_date_obj = datetime.strptime(date_str_standard, '%A %d %B %Y')

                daily_total_counts[file_date_obj] = {
                    "total_arrival_count": total_arrivals_for_day,
                    "total_depart_count": total_departures_for_day
                }

            else:
                 print(f"Could not parse date string format for '{date_str}' in file {filename}. No regex match.")


        except ValueError as e:
            print(f"Could not parse standardized date '{date_str_standard}' for file {filename}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred during date parsing for {filename}: {e}")
    else:
         print(f"Date not found in all_data for file {filename}")


# --- Calculate Average Counts Per Month ---
monthly_aggregated_counts = {}

for date_obj, counts in daily_total_counts.items():
    month_year_key = date_obj.strftime('%Y-%m')

    if month_year_key not in monthly_aggregated_counts:
        monthly_aggregated_counts[month_year_key] = {
            "total_arrivals": 0,
            "total_departures": 0,
            "number_of_days": 0
        }

    monthly_aggregated_counts[month_year_key]["total_arrivals"] += counts["total_arrival_count"]
    monthly_aggregated_counts[month_year_key]["total_departures"] += counts["total_depart_count"]
    monthly_aggregated_counts[month_year_key]["number_of_days"] += 1

monthly_average_counts = {}

for month_year_key, totals in monthly_aggregated_counts.items():
    num_days = totals["number_of_days"]
    if num_days > 0:
        avg_arrivals = totals["total_arrivals"] / num_days
        avg_departures = totals["total_departures"] / num_days
        monthly_average_counts[month_year_key] = {
            "average_arrivals": avg_arrivals,
            "average_departures": avg_departures
        }
    else:
        monthly_average_counts[month_year_key] = {
            "average_arrivals": 0,
            "average_departures": 0
        }

# --- Print Results ---

print("\n--- Daily Total SLT Counts ---")
for date_obj in sorted(daily_total_counts.keys()):
    counts = daily_total_counts[date_obj]
    print(f"{date_obj.strftime('%Y-%m-%d')}: Arrivals = {counts['total_arrival_count']}, Departures = {counts['total_depart_count']}")


print("\n--- Monthly Average SLT Counts ---")
for month_year_key in sorted(monthly_average_counts.keys()):
    averages = monthly_average_counts[month_year_key]
    year, month = map(int, month_year_key.split('-'))
    month_name = calendar.month_name[month]
    print(f"{month_name} {year}: Average Arrivals = {averages['average_arrivals']:.2f}, Average Departures = {averages['average_departures']:.2f}")


--- Daily Total SLT Counts ---

--- Monthly Average SLT Counts ---


In [ ]:
# --- Calculate Average Counts Per Year (for each year) ---

yearly_aggregated_counts = {}

for date_obj, counts in daily_total_counts.items():
    year_key = str(date_obj.year)

    if year_key not in yearly_aggregated_counts:
        yearly_aggregated_counts[year_key] = {
            "total_arrivals": 0,
            "total_departures": 0,
            "number_of_days": 0 # To count how many days contributed to the year
        }

    # Aggregate the daily totals into the yearly sum for this year
    yearly_aggregated_counts[year_key]["total_arrivals"] += counts["total_arrival_count"]
    yearly_aggregated_counts[year_key]["total_departures"] += counts["total_depart_count"]
    yearly_aggregated_counts[year_key]["number_of_days"] += 1 # Increment day count for this year

# Calculate the yearly averages for each year
yearly_average_counts_by_year = {}

for year_key, totals in yearly_aggregated_counts.items():
    num_days = totals["number_of_days"]
    if num_days > 0:
        avg_arrivals = totals["total_arrivals"] / num_days
        avg_departures = totals["total_departures"] / num_days
        yearly_average_counts_by_year[year_key] = {
            "average_arrivals": avg_arrivals,
            "average_departures": avg_departures
        }
    else:
        yearly_average_counts_by_year[year_key] = {
            "average_arrivals": 0,
            "average_departures": 0
        }


# --- Print Yearly Results (for each year) ---

print("\n--- Yearly Average SLT Counts By Year ---")
# Sort by year for chronological order
for year_key in sorted(yearly_average_counts_by_year.keys()):
    averages = yearly_average_counts_by_year[year_key]
    num_days = yearly_aggregated_counts[year_key]["number_of_days"] # Get number of days processed for this year
    if num_days > 0:
        print(f"Year {year_key} (based on {num_days} days of data):")
        print(f"  Average Arrivals Per Day = {averages['average_arrivals']:.2f}")
        print(f"  Average Departures Per Day = {averages['average_departures']:.2f}")
    else:
        print(f"Year {year_key}: No daily data available.")


--- Yearly Average SLT Counts By Year ---
Year 2025 (based on 3 days of data):
  Average Arrivals Per Day = 2.33
  Average Departures Per Day = 3.00


Code for reading in Excel Docs


In [ ]:
# Configure logging
logging.getLogger("pdfminer").setLevel(logging.ERROR) # This line is from the previous code, you can keep it or remove it if no longer needed for PDFs

# --- Data Reading from Excel ---

excel_file_path = 'your_excel_file.xlsx' # Replace with your Excel file path
all_data = {}

try:
    excel_file = pd.ExcelFile(excel_file_path)
    sheet_names = excel_file.sheet_names

    for sheet_name in sheet_names:
        print(f"Processing sheet: {sheet_name}")
        try:
            # Read the sheet into a DataFrame
            df = excel_file.parse(sheet_name, header=None) # Assuming no header in the sheets

            # Attempt to extract date from the sheet name or from a specific cell
            # You might need to adjust this based on how your date is stored in the sheet
            date_str = sheet_name # Assuming sheet name is the date string

            # You can add logic here to find the date within the sheet if needed
            # For example: date_str = df.iloc[row, col].astype(str)

            # Convert the DataFrame to a list of lists to mimic the pdfplumber table output structure
            tables_list = [df.values.tolist()] # Assuming each sheet is one "table" of data

            all_data[sheet_name] = {"date": date_str, "tables": tables_list}
            print(f"Processed sheet {sheet_name}")

        except Exception as e:
            print(f"Error processing sheet {sheet_name}: {e}")

except FileNotFoundError:
    print(f"Error: Excel file not found at {excel_file_path}")
except Exception as e:
    print(f"Error opening or reading Excel file: {e}")


# --- SLT Counting ---

slt_counts = {}

for sheet_name, data in all_data.items():
    file_date = data.get("date")

    file_tables = data.get("tables")
    print(f"Processing data from {file_date}:")

    slt_counts[sheet_name] = {}

    for i, table in enumerate(file_tables):

      df = pd.DataFrame(table)

      # Count 'SLT' in column 9 (index 8) and column 18 (index 17)
      arrival_count = 0
      depart_count = 0

      # Ensure columns exist before attempting to access them
      if len(df.columns) > 8:
          arrival_count = df[df.columns[8]].astype(str).str.lower().str.count('slt').sum()
      else:
          print(f"Column 9 (index 8) not found in sheet {sheet_name}.")


      if len(df.columns) > 17:
          depart_count = df[df.columns[17]].astype(str).str.lower().str.count('slt').sum()
      else:
          print(f"Column 18 (index 17) not found in sheet {sheet_name}.")


      # Store the counts for this specific table (sheet)
      slt_counts[sheet_name][f"sheet_{i+1}"] = {
                      "arrival_count": int(arrival_count),
                      "depart_count": int(depart_count)
                    }

      print(f"Number of rooms on and SLT for {file_date} - Arrivals: {arrival_count}, Departures: {depart_count}")


print("\n--- Final SLT Counts ---")
for sheet_name, tables_data in slt_counts.items():
  print(f"Sheet: {sheet_name}")
  for table_name, counts in tables_data.items():
    print(f"   Arrivals = {counts['arrival_count']}, Departures= {counts['depart_count']}")


# --- Calculate Total Daily Counts and Averages ---

daily_total_counts = {}

for sheet_name, tables_data in slt_counts.items():
    total_arrivals_for_day = 0
    total_departures_for_day = 0

    for table_name, counts in tables_data.items():
        total_arrivals_for_day += counts.get("arrival_count", 0)
        total_departures_for_day += counts.get("depart_count", 0)

    date_str = all_data.get(sheet_name, {}).get("date")

    if date_str:
        # Attempt to parse the date string from the sheet name or extracted date
        try:
            # You might need to adjust the date parsing based on your sheet name format
            # Example: If sheet name is 'YYYY-MM-DD'
            file_date_obj = datetime.strptime(date_str, '%Y-%m-%d') # Adjust format string as needed

            daily_total_counts[file_date_obj] = {
                "total_arrival_count": total_arrivals_for_day,
                "total_depart_count": total_departures_for_day
            }

        except ValueError as e:
            print(f"Could not parse date string '{date_str}' from sheet {sheet_name}: {e}")
        except Exception as e:
            print(f"An unexpected error occurred during date parsing for {sheet_name}: {e}")
    else:
         print(f"Date not found in all_data for sheet {sheet_name}")


# --- Calculate Average Counts Per Month ---
monthly_aggregated_counts = {}

for date_obj, counts in daily_total_counts.items():
    month_year_key = date_obj.strftime('%Y-%m')

    if month_year_key not in monthly_aggregated_counts:
        monthly_aggregated_counts[month_year_key] = {
            "total_arrivals": 0,
            "total_departures": 0,
            "number_of_days": 0
        }

    monthly_aggregated_counts[month_year_key]["total_arrivals"] += counts["total_arrival_count"]
    monthly_aggregated_counts[month_year_key]["total_departures"] += counts["total_depart_count"]
    monthly_aggregated_counts[month_year_key]["number_of_days"] += 1

monthly_average_counts = {}

for month_year_key, totals in monthly_aggregated_counts.items():
    num_days = totals["number_of_days"]
    if num_days > 0:
        avg_arrivals = totals["total_arrivals"] / num_days
        avg_departures = totals["total_departures"] / num_days
        monthly_average_counts[month_year_key] = {
            "average_arrivals": avg_arrivals,
            "average_departures": avg_departures
        }
    else:
        monthly_average_counts[month_year_key] = {
            "average_arrivals": 0,
            "average_departures": 0
        }

# --- Calculate Average Counts Per Year (for each year) ---

yearly_aggregated_counts = {}

for date_obj, counts in daily_total_counts.items():
    year_key = str(date_obj.year)

    if year_key not in yearly_aggregated_counts:
        yearly_aggregated_counts[year_key] = {
            "total_arrivals": 0,
            "total_departures": 0,
            "number_of_days": 0 # To count how many days contributed to the year
        }

    # Aggregate the daily totals into the yearly sum for this year
    yearly_aggregated_counts[year_key]["total_arrivals"] += counts["total_arrival_count"]
    yearly_aggregated_counts[year_key]["total_departures"] += counts["total_depart_count"]
    yearly_aggregated_counts[year_key]["number_of_days"] += 1 # Increment day count for this year

# Calculate the yearly averages for each year
yearly_average_counts_by_year = {}

for year_key, totals in yearly_aggregated_counts.items():
    num_days = totals["number_of_days"]
    if num_days > 0:
        avg_arrivals = totals["total_arrivals"] / num_days
        avg_departures = totals["total_departures"] / num_days
        yearly_average_counts_by_year[year_key] = {
            "average_arrivals": avg_arrivals,
            "average_departures": avg_departures
        }
    else:
        yearly_average_counts_by_year[year_key] = {
            "average_arrivals": 0,
            "average_departures": 0
        }


# --- Print Results ---

print("\n--- Daily Total SLT Counts ---")
for date_obj in sorted(daily_total_counts.keys()):
    counts = daily_total_counts[date_obj]
    print(f"{date_obj.strftime('%Y-%m-%d')}: Arrivals = {counts['total_arrival_count']}, Departures = {counts['total_depart_count']}")


print("\n--- Monthly Average SLT Counts ---")
for month_year_key in sorted(monthly_average_counts.keys()):
    averages = monthly_average_counts[month_year_key]
    year, month = map(int, month_year_key.split('-'))
    month_name = calendar.month_name[month]
    print(f"{month_name} {year}: Average Arrivals = {averages['average_arrivals']:.2f}, Average Departures = {averages['average_departures']:.2f}")


print("\n--- Yearly Average SLT Counts By Year ---")
# Sort by year for chronological order
for year_key in sorted(yearly_average_counts_by_year.keys()):
    averages = yearly_average_counts_by_year[year_key]
    num_days = yearly_aggregated_counts[year_key]["number_of_days"] # Get number of days processed for this year
    if num_days > 0:
        print(f"Year {year_key} (based on {num_days} days of data):")
        print(f"  Average Arrivals Per Day = {averages['average_arrivals']:.2f}")
        print(f"  Average Departures Per Day = {averages['average_departures']:.2f}")
    else:
        print(f"Year {year_key}: No daily data available.")

Error: Excel file not found at your_excel_file.xlsx

--- Final SLT Counts ---

--- Daily Total SLT Counts ---

--- Monthly Average SLT Counts ---

--- Yearly Average SLT Counts By Year ---
